In [1]:
!pip install torch --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
!pip install setfit datasets evaluate kagglehub wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 7.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch.nn as nn
from sklearn.model_selection import train_test_split
from datasets import Dataset
from datasets import concatenate_datasets
from datasets import DatasetDict
import evaluate

from setfit import SetFitModel, SetFitTrainer
import pandas as pd
import os
os.environ["KAGGLE_USERNAME"] = "USERNAME"
os.environ["KAGGLE_KEY"] = "KAGGLE_KEY"
import kagglehub

In [2]:
import wandb
wandb.init(mode="disabled")

/usr/local/lib/python3.10/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)


In [5]:
# Download the latest version.
dataset_path = kagglehub.dataset_download("anonymous/wolbanking77v1")
print(dataset_path)

100%|██████████| 1.02M/1.02M [00:00<00:00, 1.30MB/s]

Extracting files...
/root/.cache/kagglehub/datasets/anonymous/wolbanking77v1/versions/4


In [ ]:
from huggingface_hub import login

login(
  token="HF_TOKEN", # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [4]:
df_train = pd.read_csv(
    "/root/.cache/kagglehub/datasets/anonymous/wolbanking77v1/versions/4/wolbanking77v1/full/train/train.csv")

df_test = pd.read_csv(
    "/root/.cache/kagglehub/datasets/anonymous/wolbanking77v1/versions/4/wolbanking77v1/full/test/test.csv")

df_train = df_train[['input_wo', 'label']]
df_test = df_test[['input_wo', 'label']]

#removal of punctuation marks
df_train['input_wo_clean'] = df_train['input_wo'].replace(regex=True, to_replace = '[^\\w\\s]', value = ' ')
df_test['input_wo_clean'] = df_test['input_wo'].replace(regex=True, to_replace = '[^\\w\\s]', value = ' ')

# Apply lowercase
df_train['input_wo_clean'] = df_train['input_wo_clean'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df_test['input_wo_clean'] = df_test['input_wo_clean'].apply(lambda x: " ".join(x.lower() for x in x.split()))

train_ds = Dataset.from_pandas(df_train[['input_wo_clean', 'label']], split="train")
test_ds = Dataset.from_pandas(df_test[['input_wo_clean', 'label']], split="test")

raw_dataset = DatasetDict({"train": train_ds, "test": test_ds})
# raw_dataset = raw_dataset.remove_columns("__index_level_0__")
raw_dataset =  raw_dataset.rename_column("input_wo_clean", "text") # to match Trainer
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 7832
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1959
    })
})

In [5]:
def compute_metrics(y_true, y_pred, average="weighted"):
    f1 = evaluate.load("f1")
    precision = evaluate.load("precision")
    recall = evaluate.load("recall")
    return {"f1_score" : f1.compute(references=y_true, predictions=y_pred, average=average),
            "precision_score" : precision.compute(references=y_true, predictions=y_pred, average=average),
            "recall_score" : recall.compute(references=y_true, predictions=y_pred, average=average)
    }

# bert-base-uncased

In [6]:
# Load SetFit model from Hub
model = SetFitModel.from_pretrained("bert-base-uncased")
model.to("cuda")

No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [7]:
# f1 = evaluate.load("f1")
from setfit import sample_dataset

raw_dataset = raw_dataset.class_encode_column("label")

Casting to class labels:   0%|          | 0/7832 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/1959 [00:00<?, ? examples/s]

In [8]:
train_dataset = sample_dataset(raw_dataset["train"], label_column="label", num_samples=8) # 2 shots or 8 shots
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 616
})

In [9]:
test_dataset = raw_dataset["test"]
test_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 1959
})

In [10]:
model.labels = list(set(train_ds['label']))

In [11]:
from setfit import TrainingArguments, Trainer

args = TrainingArguments(
    batch_size=64, # 32 64 96
    num_epochs=1,
    save_strategy="epoch",
    report_to="none",
)

In [12]:
# Preparing the trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    metric=compute_metrics, #"f1",
    metric_kwargs={"average": "weighted"},
)

# os.environ["WANDB_DISABLED"] = "true"

Map:   0%|          | 0/616 [00:00<?, ? examples/s]

In [13]:
trainer.train()

***** Running training *****
  Num unique pairs = 374528
  Batch size = 64
  Num epochs = 1


Step,Training Loss
1,0.317200
50,0.296500
100,0.220800


KeyboardInterrupt: 

In [50]:
# trainer.metric = compute_metrics

In [15]:
trainer.evaluate(test_dataset)

***** Running evaluation *****


{'f1_score': {'f1': 0.07507905615896235},
 'precision_score': {'precision': 0.08969857821270721},
 'recall_score': {'recall': 0.07401735579377233}}

# Davlan/afro-xlmr-large

In [20]:
# Load SetFit model from Hub
model = SetFitModel.from_pretrained("Davlan/afro-xlmr-large")
model.to("cuda")

No sentence-transformers model found with name Davlan/afro-xlmr-large. Creating a new one with mean pooling.
Some weights of XLMRobertaModel were not initialized from the model checkpoint at Davlan/afro-xlmr-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [21]:
raw_dataset = raw_dataset.class_encode_column("label")

Casting to class labels:   0%|          | 0/4000 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [25]:
from setfit import sample_dataset

train_dataset = sample_dataset(raw_dataset["train"], label_column="label", num_samples=8) # 8
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 616
})

In [26]:
test_dataset = raw_dataset["test"]
test_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})

In [27]:
model.labels = list(set(train_ds['label']))

In [28]:
from setfit import TrainingArguments, Trainer

args = TrainingArguments(
    batch_size=16, # 32 64 96
    num_epochs=1,
    save_strategy="epoch",
    # report_to="tensorboard",
)

In [29]:
# Preparing the trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    metric=compute_metrics,
    metric_kwargs={"average": "weighted"},
)

Map:   0%|          | 0/616 [00:00<?, ? examples/s]

In [30]:
trainer.train()

***** Running training *****
  Num unique pairs = 374528
  Batch size = 16
  Num epochs = 1


Step,Training Loss
1,0.239700
50,0.326800
100,0.322000
150,0.275000
200,0.258700
250,0.240100
300,0.246300
350,0.246700
400,0.235600
450,0.228000


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [31]:
trainer.evaluate(test_dataset)

***** Running evaluation *****


{'f1_score': {'f1': 0.4722238211454406},
 'precision_score': {'precision': 0.5120543698776284},
 'recall_score': {'recall': 0.47}}

# bonadossou/afrolm_active_learning

USING A100 PCIe GPU

In [9]:
!pip install protobuf
!pip install sentencepiece
!pip install tiktoken


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.6 MB/s eta 0:00:00a 0:00:010m

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [6]:
# Load SetFit model from Hub
model = SetFitModel.from_pretrained("bonadossou/afrolm_active_learning")
model.to("cuda")

No sentence-transformers model found with name bonadossou/afrolm_active_learning. Creating a new one with mean pooling.
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of XLMRobertaModel were not initialized from the model checkpoint at bonadossou/afrolm_active_learning and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [7]:
# f1 = evaluate.load("f1")
raw_dataset = raw_dataset.class_encode_column("label")

Casting to class labels:   0%|          | 0/4000 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
from setfit import sample_dataset

train_dataset = sample_dataset(raw_dataset["train"], label_column="label", num_samples=8) # 8
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 616
})

In [9]:
test_dataset = raw_dataset["test"]
test_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})

In [10]:
model.labels = list(set(train_ds['label']))

In [11]:
from setfit import TrainingArguments, Trainer

args = TrainingArguments(
    batch_size=64, # 32 64 96 112
    num_epochs=1,
    save_strategy="epoch",
)

In [12]:
# Preparing the trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    metric=compute_metrics, #"f1",
    metric_kwargs={"average": "weighted"},
)

Map:   0%|          | 0/616 [00:00<?, ? examples/s]

In [13]:
trainer.train()

***** Running training *****
  Num unique pairs = 374528
  Batch size = 64
  Num epochs = 1


Step,Training Loss
1,0.276500
50,0.269900
100,0.245200
150,0.229000
200,0.203400
250,0.184600
300,0.171700
350,0.153600
400,0.133700
450,0.116900


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:393: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 256}
  warnings.warn(


The OrderedVocab you are attempting to save contains holes for indices [3], your vocabulary could be corrupted !
The OrderedVocab you are attempting to save contains holes for indices [3], your vocabulary could be corrupted !


In [14]:
trainer.evaluate(test_dataset)

***** Running evaluation *****


{'f1_score': {'f1': 0.4134168055434866},
 'precision_score': {'precision': 0.4565793498812849},
 'recall_score': {'recall': 0.418}}

# MoritzLaurer/mDeBERTa-v3-base-mnli-xnli

In [6]:
# Load SetFit model from Hub
model = SetFitModel.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
model.to("cuda")

No sentence-transformers model found with name MoritzLaurer/mDeBERTa-v3-base-mnli-xnli. Creating a new one with mean pooling.
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [7]:
# f1 = evaluate.load("f1")
raw_dataset = raw_dataset.class_encode_column("label")

Casting to class labels:   0%|          | 0/4000 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
from setfit import sample_dataset

train_dataset = sample_dataset(raw_dataset["train"], label_column="label", num_samples=8) # 8
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 616
})

In [9]:
test_dataset = raw_dataset["test"]
test_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})

In [10]:
model.labels = list(set(train_ds['label']))

In [11]:
from setfit import TrainingArguments, Trainer

args = TrainingArguments(
    batch_size=32, # 32 64 96 112
    num_epochs=1,
    save_strategy="epoch",
)

# Preparing the trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    metric=compute_metrics, #"f1",
    metric_kwargs={"average": "weighted"},
)

Map:   0%|          | 0/616 [00:00<?, ? examples/s]

In [12]:
trainer.train()

***** Running training *****
  Num unique pairs = 374528
  Batch size = 32
  Num epochs = 1


Step,Training Loss
1,0.448800
50,0.417300
100,0.303300
150,0.286300
200,0.266200
250,0.252700
300,0.236600
350,0.220100
400,0.220300
450,0.198100


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
trainer.evaluate(test_dataset)

***** Running evaluation *****


{'f1_score': {'f1': 0.351831693052262},
 'precision_score': {'precision': 0.4025673552754435},
 'recall_score': {'recall': 0.349}}

# AfritevaV2

In [6]:
# Load SetFit model from Hub
model = SetFitModel.from_pretrained("castorini/afriteva_v2_base")
model.to("cuda")

No sentence-transformers model found with name castorini/afriteva_v2_base. Creating a new one with mean pooling.
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-pa

In [7]:
raw_dataset = raw_dataset.class_encode_column("label")

Casting to class labels:   0%|          | 0/4000 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
from setfit import sample_dataset

train_dataset = sample_dataset(raw_dataset["train"], label_column="label", num_samples=8) # 2 - 8
test_dataset = raw_dataset["test"]

In [9]:
model.labels = list(set(train_ds['label']))

In [10]:
from setfit import TrainingArguments, Trainer

args = TrainingArguments(
    batch_size=32, # 32 64 96 112
    num_epochs=1,
    save_strategy="epoch",
)

# Preparing the trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    metric=compute_metrics, #"f1",
    metric_kwargs={"average": "weighted"},
)

Map:   0%|          | 0/616 [00:00<?, ? examples/s]

In [11]:
trainer.train()

***** Running training *****
  Num unique pairs = 374528
  Batch size = 32
  Num epochs = 1


Step,Training Loss
1,0.247000
50,0.255400
100,0.260300
150,0.259000
200,0.244600
250,0.245800
300,0.244700
350,0.237800
400,0.238100
450,0.237500


In [12]:
trainer.evaluate(test_dataset)

***** Running evaluation *****


{'f1_score': {'f1': 0.2822005613786148},
 'precision_score': {'precision': 0.31061411014166435},
 'recall_score': {'recall': 0.282}}